In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
189,AFG,Asia,Afghanistan,2020-07-17,35070.0,76.0,1115.0,21.0,900.886,1.952,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
320,ALB,Europe,Albania,2020-07-17,3851.0,99.0,104.0,3.0,1338.175,34.401,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
515,DZA,Africa,Algeria,2020-07-17,21355.0,585.0,1052.0,12.0,486.990,13.341,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-16,France,173529.0,0.0,30135.0,0.0,0.0,0.0
0,2020-07-16,China,85307.0,0.0,4653.0,0.0,0.0,0.0
0,2020-07-16,Italy,243759.0,0.0,35037.0,0.0,0.0,0.0
0,2020-07-16,Spain,258173.0,0.0,28428.0,0.0,0.0,0.0
0,2020-07-16,United States,3557378.0,0.0,138368.0,0.0,0.0,0.0
0,2020-07-16,World,13776618.0,0.0,590122.0,0.0,0.0,0.0
0,2020-07-16,United Kingdom,293180.0,0.0,45228.0,0.0,0.0,0.0
0,2020-07-16,Germany,201032.0,0.0,9093.0,0.0,0.0,0.0
0,2020-07-16,Iran,267962.0,0.0,13572.0,0.0,0.0,0.0
0,2020-07-16,Turkey,216890.0,0.0,5438.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-17,France,173529.0,173838.0,30135.0,30138.0,-309.0,-3.0
0,2020-07-17,China,85307.0,85323.0,4653.0,4644.0,-16.0,9.0
0,2020-07-17,Italy,243759.0,243736.0,35037.0,35017.0,23.0,20.0
0,2020-07-16,Spain,258173.0,258855.0,28428.0,28416.0,-682.0,12.0
0,2020-07-17,United States,3557378.0,3576221.0,138368.0,138358.0,-18843.0,10.0
0,2020-07-17,World,13776618.0,13788300.0,590122.0,589688.0,-11682.0,434.0
0,2020-07-17,United Kingdom,293180.0,292552.0,45228.0,45119.0,628.0,109.0
0,2020-07-17,Germany,201032.0,200843.0,9093.0,9082.0,189.0,11.0
0,2020-07-17,Iran,267962.0,267061.0,13572.0,13608.0,901.0,-36.0
0,2020-07-17,Turkey,216890.0,216873.0,5438.0,5440.0,17.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")